In [1]:
import asyncio
import aiohttp
import time
import nest_asyncio
import pandas as pd 
import numpy as np 

In [ ]:
l1=pd.read_parquet('./EnglandLinks.parquet')
l2=pd.read_parquet('./FrenchLinks.parquet')
l3=pd.read_parquet('./GermanLinks.parquet')
l4=pd.read_parquet('./ItalianLinks.parquet')
l5=pd.read_parquet('./SpainLinks.parquet')
l1.columns={'1'}
l2.columns={'1'}
l3.columns={'1'}
l4.columns={'1'}
l5.columns={'1'}
finalLinks = pd.concat([l1,l2,l3,l4,l5],axis=0)
finalLinks.reset_index(drop=True,inplace=True)
listman =[]
for i in range(len(finalLinks)):
    listman.append(finalLinks['1'].iloc[i])


In [ ]:
nest_asyncio.apply()


playersLink=[]
index=0
async def get(url, session):
    try:
        async with session.get(url=url) as response:
            resp = await response.read()
            pageSoup = BeautifulSoup(resp,'html.parser')
            players = pageSoup.find_all("td",{"class" : "hauptlink"})
            i,j=0,0
            while (i<len(players)):
                try:
                    playersLink.append(players[i].a.get('href'))
                except:
                    playersLink.append(np.nan)
                i+=2
            # print(url)

    except Exception as e:
        print("Unable to get url {} due to {}.".format(url, e.__class__))


async def main(urls):
    async with aiohttp.ClientSession() as session:
        ret = await asyncio.gather(*[get(url, session) for url in urls])
    print("Finalized all. Return is a list of len {} outputs.".format(len(ret)))
    

urls = listman
start = time.time()
asyncio.run(main(urls))
end = time.time()

print("Took {} seconds to pull {} websites.".format(end - start, len(urls)))


df = pd.DataFrame({'PlayersLink':playersLink})
df.to_parquet('allPlayersLink.parquet',engine='pyarrow')